In [1]:
import matplotlib.pyplot as plt

In [2]:
import torch
import numpy as np

## Creating inputs and outputs

Taking 5 rows of inputs having 3 features, outputs having 2 variables.
So, inputs are of shape 5x3 and outputs of shape 5x2.

In [3]:
inputs = np.array([[2, 50, -20],
              [10, 230, -110],
              [5, 120, -48],
              [7, 180, -80],
              [12, 300, -140]
             ], dtype='float32')

print(inputs)

[[   2.   50.  -20.]
 [  10.  230. -110.]
 [   5.  120.  -48.]
 [   7.  180.  -80.]
 [  12.  300. -140.]]


In [4]:
targets = np.array([[100, 300],
              [900, 2800],
              [430, 1200],
              [650, 1700],
              [1320, 4000]
             ], dtype='float32')

print(targets)

[[ 100.  300.]
 [ 900. 2800.]
 [ 430. 1200.]
 [ 650. 1700.]
 [1320. 4000.]]


### Converting into torch.tensor objects

In [5]:
x = torch.from_numpy(inputs)
y = torch.from_numpy(targets)

## Building model from scratch

### Initializing weights and bias

In [6]:
w = torch.randn(y.shape[1],x.shape[1], requires_grad=True)
b = torch.randn(y.shape[1], requires_grad=True)

In [7]:
print(w,'\n')
print(b)

tensor([[0.0666, 0.8153, 0.3651],
        [1.1149, 0.4779, 0.4291]], requires_grad=True) 

tensor([-0.2475,  0.1149], requires_grad=True)


### Defining model

In [8]:
def model(x, w, b):
    return(x @ w.t() + b)

In [9]:
## Initial values

y_pred = model(x, w, b)

print("Target values :\n",y,'\n')
print("Predicted values :\n",y_pred,'\n')

Target values :
 tensor([[ 100.,  300.],
        [ 900., 2800.],
        [ 430., 1200.],
        [ 650., 1700.],
        [1320., 4000.]]) 

Predicted values :
 tensor([[ 33.3482,  17.6571],
        [147.7745,  73.9774],
        [ 80.3954,  42.4391],
        [117.7630,  59.6112],
        [194.0249,  96.7865]], grad_fn=<AddBackward0>) 



### Loss function

In [10]:
def mse(y_pred, y):
    diff = y_pred - y
    return(torch.sum(diff**2)/diff.numel())

In [11]:
loss = mse(y_pred, y)
print(loss)

tensor(2902042.2500, grad_fn=<DivBackward0>)


### Computing gradients using autograd

In [12]:
loss.backward()

In [13]:
print(w.grad)
print(b.grad)

tensor([[  -5328.1885, -130378.4453,   60214.8672],
        [ -18386.8008, -449248.7500,  207750.6562]])
tensor([ -565.3388, -1941.9059])


### Adjusting weights and biases as per gradients

In [14]:
lr = 0.00001

In [15]:
with torch.no_grad():
    w -= w.grad * lr
    b -= b.grad * lr
    
    # Setting gradients to zero
    
    w.grad.zero_()
    b.grad.zero_()

### Check new loss

In [16]:
y_pred = model(x, w, b)
loss_new = mse(y_pred, y)
print(loss)
print(loss_new)

tensor(2902042.2500, grad_fn=<DivBackward0>)
tensor(862654.3750, grad_fn=<DivBackward0>)


### Training for multiple epochs

In [17]:
epochs = 10000

In [18]:
for i in range(epochs):
    y_pred = model(x, w, b)
    loss = mse(y_pred, y)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * lr
        b -= b.grad * lr
        w.grad.zero_()
        b.grad.zero_()

In [19]:
y_pred = model(x, w, b)

print("Target values :\n",y,'\n')
print("Predicted values :\n",y_pred,'\n')

Target values :
 tensor([[ 100.,  300.],
        [ 900., 2800.],
        [ 430., 1200.],
        [ 650., 1700.],
        [1320., 4000.]]) 

Predicted values :
 tensor([[ 163.3475,  424.7473],
        [ 971.6277, 2979.9382],
        [ 403.3122, 1051.1436],
        [ 696.8544, 2010.1288],
        [1237.0767, 3714.6946]], grad_fn=<AddBackward0>) 



## Building model using torch inbuilts

### Dataset and Dataloader

In [20]:
train_dataset = torch.utils.data.TensorDataset(x, y)
train_dataset[:3]

(tensor([[   2.,   50.,  -20.],
         [  10.,  230., -110.],
         [   5.,  120.,  -48.]]), tensor([[ 100.,  300.],
         [ 900., 2800.],
         [ 430., 1200.]]))

In [21]:
batch_size = 5
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle = True)

for xb, yb in train_dataloader:
    print(xb)
    print(yb)
    break

tensor([[   5.,  120.,  -48.],
        [  10.,  230., -110.],
        [   2.,   50.,  -20.],
        [   7.,  180.,  -80.],
        [  12.,  300., -140.]])
tensor([[ 430., 1200.],
        [ 900., 2800.],
        [ 100.,  300.],
        [ 650., 1700.],
        [1320., 4000.]])


### torch.nn.Linear
This will initialize weights and biases automatically.

In [22]:
model = torch.nn.Linear(3,2)

print(model.weight,'\n', model.bias)

Parameter containing:
tensor([[ 0.3546,  0.3004,  0.0953],
        [-0.3937,  0.3003, -0.3564]], requires_grad=True) 
 Parameter containing:
tensor([0.4044, 0.4853], requires_grad=True)


To view all parameters, we can use ```model.parameters()``` method.

In [23]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3546,  0.3004,  0.0953],
         [-0.3937,  0.3003, -0.3564]], requires_grad=True),
 Parameter containing:
 tensor([0.4044, 0.4853], requires_grad=True)]

Making predictions using initialized model:

In [24]:
y_pred = model(x)
print(y_pred)

tensor([[ 14.2283,  21.8399],
        [ 62.5634, 104.8165],
        [ 33.6527,  51.6575],
        [ 49.3374,  80.2916],
        [ 81.4426, 135.7406]], grad_fn=<AddmmBackward>)


### Loss function

In [25]:
mse = torch.nn.functional.mse_loss

loss = mse(y_pred, y)
print(loss)

tensor(2897660.2500, grad_fn=<MseLossBackward>)


### Optimizer

In [26]:
sgd = torch.optim.SGD(model.parameters(), lr=0.00001)

To automatically update the parameters (weights and biases), we'll use ```sgd.step()``` function.

### Training the model

Here, we'll work batches of data instead of processing the entire training data in every iteration.

In [27]:
# Utility function to train the model

def fit(epochs, model, loss_function, optimizer, dataloader):
    for i in range(epochs):
        for xb, yb in dataloader:
            
            # Make prediction
            y_pred = model(xb)
            
            # Calculate loss
            loss = loss_function(y_pred, yb)

            # Compute gradients
            loss.backward()
            
            # Update parameters using gradients
            optimizer.step()
            
            # Reset gradients to zero
            optimizer.zero_grad()

In [28]:
fit(10000, model, mse, sgd, train_dataloader)

In [30]:
y_pred = model(x)

print("Target values :\n",y,'\n')
print("Predicted values :\n",y_pred,'\n')

Target values :
 tensor([[ 100.,  300.],
        [ 900., 2800.],
        [ 430., 1200.],
        [ 650., 1700.],
        [1320., 4000.]]) 

Predicted values :
 tensor([[ 173.7973,  466.7914],
        [ 966.2294, 2940.4749],
        [ 419.6322, 1129.8903],
        [ 701.8192, 2040.2812],
        [1230.2186, 3689.6101]], grad_fn=<AddmmBackward>) 

